# Chassis.ml demo

## Easily build MLflow models into {KFServing, Modzy} Docker images

This demo will show you how we can train a model, define custom pre- and post-processing steps, save it in MLflow format and then build it into a container image and push it to docker hub with a single command.

By easily connecting MLflow models to Docker images with a simple Python SDK for data scientists & ML engineers, Chassis is the missing link between MLflow and DevOps.

This demo can be run in local using minikube and a local installation of Chassis.

## Prerequisites

* [Docker Hub](https://hub.docker.com/) account (free one is fine)
* The browser you're reading this in :-)
* Existing local installation of Chassis

In [1]:
import chassisml
import mlflow.pyfunc
from joblib import dump, load

### Load model

Load pretrained PyTorch ResNet50 model and ImageNet labels.

The goal for Chassis service is to create an image that exposes this model.

In [3]:
import pickle
import torchvision.models as models

model = models.resnet50(pretrained=True)
labels = pickle.load(open('./modzy/imagenet_labels.pkl','rb'))

In [30]:
# Wrap your model in a pyfunc and provide auxiliary functionality through extension of the
# mlflow PythonModel class with methods pre_process, post_process, and explain

# Must inherit from mlflow.pyfunc.PythonModel
class CustomModel(mlflow.pyfunc.PythonModel):
    
    def load_context(self, context):
        """
        # This method is REQUIRED
        # Load anything that needs to persist across inference runs here
        """

        import torch
        from torchvision import transforms
        
        # Note that the model and labels were loaded outside of this class in the previous cell
        # They were simply assigned to instance variables here
        # This is allowed
        self.model = model
        self.model.eval()
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.labels = labels

        self.transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])        

    def predict(self, context, input_dict): 
        """
        This method is REQUIRED.
        When an inference job comes in, this will be executed.
        input_dict['input_data_bytes'] will contain the input file in bytes format.
        This method must those bytes, running inference, postprocess if needed, and return results.
        In this example, we have broken out preprocessing and postproceessing into their own methods.
        However, if you'd like, you can handle everything within this predict() method.
        """
        preprocessed_input = self.preprocess(input_dict['input_data_bytes'])
        inference_results = self.model(preprocessed_input)
        return self.postprocess(inference_results)

    def preprocess(self, img_bytes):
        import cv2
        import torch
        import numpy as np

        # convert input bytes into image NumPy array 
        decoded = cv2.imdecode(np.frombuffer(img_bytes, np.uint8), -1)
        img_t = self.transform(decoded)
        batch_t = torch.unsqueeze(img_t, 0).to(self.device)
        return batch_t
    
    def postprocess(self, predictions):
        # postprocess model output into desired output format

        import torch
        import torch.nn as nn
        
        percentage = torch.nn.functional.softmax(predictions, dim=1)[0]

        _, indices = torch.sort(predictions, descending=True)
        inference_result = {
            "classPredictions": [
                {"class": self.labels[idx.item()], "score": percentage[idx].item()}
            for idx in indices[0][:5] ]
        }
                
        structured_output = {
            "data": {
                "result": inference_result,
                "explanation": None,
                "drift": None,
            }
        }
        return structured_output

In [31]:
# Define conda environment with all required dependencies for your model
# mlflow is ALWAYS REQUIRED

conda_env = {
    "channels": ["defaults", "conda-forge", "pytorch"],
    "dependencies": [
        "python=3.8.5",
        "pytorch",
        "torchvision",
        "pip",
        {
            "pip": [
                "mlflow",
                "opencv-python-headless"
            ],
        },
    ],
    "name": "torch_env"
}

### Save the model

Transform the model into MLFlow format.

In [35]:
!rm -rf mlflow_custom_pyfunc_torch
model_save_path = "mlflow_custom_pyfunc_torch"
mlflow.pyfunc.save_model(path=model_save_path, python_model=CustomModel(), conda_env=conda_env)

Load the MLFlow model and test it.

In [36]:
import json

# The model's predict() method is expecting a dict where:
# input_dict['input_data_bytes'] will contain the input file in bytes format
# Let's mimic that here to test our saved model's functionality with an example image
input_dict = {'input_data_bytes': open('./modzy/airplane.jpg','rb').read()}

classifier = mlflow.pyfunc.load_model(model_save_path)
predictions = classifier.predict(input_dict)
print(json.dumps(predictions, indent=4))

{
    "data": {
        "result": {
            "classPredictions": [
                {
                    "class": "airliner",
                    "score": 0.919350802898407
                },
                {
                    "class": "wing",
                    "score": 0.05414900183677673
                },
                {
                    "class": "warplane, military plane",
                    "score": 0.010522871278226376
                },
                {
                    "class": "aircraft carrier, carrier, flattop, attack aircraft carrier",
                    "score": 0.004400244448333979
                },
                {
                    "class": "crane",
                    "score": 0.001940063084475696
                }
            ]
        },
        "explanation": null,
        "drift": null
    }
}


We check that the model has been correctly saved inside the `model` directory.

In [34]:
!ls ./mlflow_custom_pyfunc_torch

MLmodel          conda.yaml       python_model.pkl requirements.txt


### Get Docker Hub credentials securely

Now we prompt the user (you!) for your docker hub username and password in such a way that the value itself doesn't get written into the notebook, which is sensible security best-practice.

In [10]:
import getpass
import base64
username = getpass.getpass('docker hub username')
password = getpass.getpass('docker hub password')

### Complete the data

Now we can construct the metadata that the chassis service needs to build and publish the container to docker hub. In case `publish` is `False` the image is not uploaded to Docker Hub.

In [11]:
image_data = {
    'name': f'{username}/chassisml-pytorch-ic:latest',
    'version': '0.0.1',
    'model_name': 'imagenet',
    'model_path': './mlflow_custom_pyfunc_torch',
    'registry_auth': base64.b64encode(f"{username}:{password}".encode("utf-8")).decode("utf-8"),
    'publish': True
}

#### Complete Modzy data

In the case of KFServing, no more data is required. When it comes to Modzy, we will need to define some more data:

* `metadata_path`: this is the path to the [model.yaml](https://models.modzy.com/docs/model-packaging/model-packaging-python-template/yaml-file_) file that is needed to define all information about the model. Chassis has a default one, but you should define your own based on [this example](https://github.com/modzy/chassis/blob/main/chassisml-sdk/examples/modzy/model.yaml)
* `sample_input_path`: this is the path to the [sample input](https://models.modzy.com/docs/model-deployment/model-deployment/input-outputs) that is needed when deploying the model. An example can be found [here](https://github.com/modzy/chassis/blob/main/chassisml-sdk/examples/modzy/input_sample.json)
* `deploy`: if it is `True` Chassis will manage to deploy the model into Modzy platform. Otherwise you can do this manually through the Modzy UI
* `api_key`: you should have your own [api key](https://models.modzy.com/docs/how-to-guides/api-keys) from Modzy in order to let Chassis deploy the model for you

Notice that if `deploy` is False this means that you can avoid defining the rest of the fields. Anyway, `metadata_path` should be defined in case you will eventually deploy the model to Modzy. This is important because the model will use this information when being deployed to Modzy, so it needs to be updated.

In [12]:
import getpass
modzy_api_key = getpass.getpass('modzy api_key')

In [13]:
modzy_data = {
    'metadata_path': './modzy/model-torch.yaml',
    'sample_input_path': './modzy/airplane.jpg',
    'deploy': True,
    'api_key': modzy_api_key
}

### Forward ports to access service and registry

This assumes that you are running these commands on your own terminal to redirect the service (port 5000) and the registry (port 5001) to localhost.

In [13]:
! # kubectl port-forward service/chassis 5000:5000

### Launch the job

Important fields that we should fill in here are:

* `module`: library that has been used to create the model
* `image_data`: the values defined above
* `image_type`: this is needed in case we are training images so afterwards the proxy will know how to interpret data
* `base_url`: point to your running Chassis service here

In [44]:
# base_url must point to a running Chassis instance
# in this example, we assume that Chassis is running locally
res = chassisml.publish(
    image_data=image_data,
    modzy_data=modzy_data,
    base_url='http://localhost:5000'
)

error = res.get('error')
job_id = res.get('job_id')

if error:
    print('Error:', error)
else:
    print('Job ID:', job_id)

Building image... Ok!
Job ID: chassis-builder-job-d9830bd0-91fd-4c6c-b669-0e6a73847ce8


After the request is made, Chassis launches a job that runs Kaniko and builds the docker image based on the values provided.

You can get the id of the job created from the result of the request. This id can be used to ask for the status of the job.

This is an example of the data that is shown when the job has not finished yet.

In [55]:
chassisml.get_job_status(job_id)

{'result': {'containerImage': {'containerImageSize': 0,
   'loadPercentage': 10,
   'loadStatus': 'IN_PROGRESS',
   'repositoryName': 'vpf0yrzjle',
   'uploadPercentage': 0,
   'uploadStatus': 'IN_PROGRESS'},
  'container_url': 'https://integration.modzy.engineering/models/vpf0yrzjle/0.1.0',
  'createdAt': '2021-11-16T23:12:53.135+00:00',
  'inputValidationSchema': '',
  'inputs': [{'acceptedMediaTypes': 'image/jpeg',
    'description': 'input RGB jpeg image',
    'maximumSize': 5000000,
    'name': 'input.jpg'}],
  'isActive': False,
  'isAvailable': True,
  'longDescription': 'It classifies images.',
  'model': {'author': 'Integration',
   'createdByEmail': 'saumil.dave@modzy.com',
   'description': 'This is an image built by chassis that exposes a PyTorch model.',
   'features': [],
   'isActive': False,
   'isCommercial': False,
   'isRecommended': False,
   'latestActiveVersion': '',
   'latestVersion': '0.1.0',
   'modelId': 'vpf0yrzjle',
   'name': 'chassis-torch-ic',
   'permal

And this is an example of the data that is shown when the job has already finished.

Two top keys can be seen:

* `result`: in case `deploy` was `True` this will contain some information related to the model deployed in Modzy. In particular we can see the full url to the model if we access the `container_url` key.
* `status`: this contains the information about the kubernetes job that has built the image (and uploaded it to Modzy in case `deploy` was `True` as explained above)

In [56]:
job_status = chassisml.get_job_status(job_id)
result = job_status.get('result')

job_status


{'result': {'containerImage': {'containerImageSize': 0,
   'loadPercentage': 10,
   'loadStatus': 'IN_PROGRESS',
   'repositoryName': 'vpf0yrzjle',
   'uploadPercentage': 0,
   'uploadStatus': 'IN_PROGRESS'},
  'container_url': 'https://integration.modzy.engineering/models/vpf0yrzjle/0.1.0',
  'createdAt': '2021-11-16T23:12:53.135+00:00',
  'inputValidationSchema': '',
  'inputs': [{'acceptedMediaTypes': 'image/jpeg',
    'description': 'input RGB jpeg image',
    'maximumSize': 5000000,
    'name': 'input.jpg'}],
  'isActive': False,
  'isAvailable': True,
  'longDescription': 'It classifies images.',
  'model': {'author': 'Integration',
   'createdByEmail': 'saumil.dave@modzy.com',
   'description': 'This is an image built by chassis that exposes a PyTorch model.',
   'features': [],
   'isActive': False,
   'isCommercial': False,
   'isRecommended': False,
   'latestActiveVersion': '',
   'latestVersion': '0.1.0',
   'modelId': 'vpf0yrzjle',
   'name': 'chassis-torch-ic',
   'permal

### Inference!

Now that the model has been deployed into Modzy platform, we can make a request against it to see it working. 

This is going to use the sample input defined above, which is going to be wrapped in a [Modzy Job](https://models.modzy.com/docs/jobs/jobs/submit-job-text). Take into account that input names must match model input filenames.

In [57]:
import base64

input_name = result['inputs'][0]['name'] # e.g. input.json
input_b64 = base64.b64encode(open('./modzy/airplane.jpg','rb').read()).decode('ascii')

request_data = {
  'model': {
    'identifier': f'{result.get("model").get("modelId")}',
    'version': f'{result.get("version")}'
  },
  'input': {
    'type': 'embedded',
    'sources': {
      'input': {
        input_name: f'data:image/jpeg;base64,{input_b64}'
        # ^ this has to match the input filename specified when the model was created
      }
    }
  }
}

import requests

res_job = requests.post(
    'https://integration.modzy.engineering/api/jobs',
    json=request_data,
    headers={'Authorization': f'ApiKey {modzy_api_key}'}
)

res_job_json = res_job.json()

res_job_json

{'model': {'identifier': 'vpf0yrzjle',
  'version': '0.1.0',
  'name': 'chassis-torch-ic'},
 'status': 'SUBMITTED',
 'totalInputs': 1,
 'jobIdentifier': '1c8f2aca-ca98-4311-b231-2737066aac1d',
 'accessKey': 'q6ED7pBAFtCsDkbaBLpt',
 'explain': False,
 'jobType': 'batch',
 'accountIdentifier': 'modzy-account',
 'team': {'identifier': '830b2012-1557-48a9-a4df-d867b5a0939a'},
 'user': {'identifier': 'eb8cf7ea-3930-410d-812f-7430e6baf3c5',
  'externalIdentifier': 'saumil.dave@modzy.com',
  'firstName': 'Saumil',
  'lastName': 'Dave',
  'email': 'saumil.dave@modzy.com',
  'status': 'active'},
 'jobInputs': {'identifier': ['input']},
 'submittedAt': '2021-11-16T23:17:51.689+00:00',
 'hoursDeleteInput': 24,
 'imageClassificationModel': True}

We must wait until the job has finished and we can see the results.

In [59]:
job_id = res_job_json.get('jobIdentifier')

res_result = requests.get(
    f'https://integration.modzy.engineering/api/results/{job_id}',
    headers={'Authorization': f'ApiKey {modzy_api_key}'}
)

res_result_json = res_result.json()

res_result_json

{'jobIdentifier': '1c8f2aca-ca98-4311-b231-2737066aac1d',
 'accountIdentifier': 'modzy-account',
 'team': {'identifier': '830b2012-1557-48a9-a4df-d867b5a0939a'},
 'total': 1,
 'completed': 1,
 'failed': 0,
 'finished': True,
 'submittedByKey': 'q6ED7pBAFtCsDkbaBLpt',
 'explained': False,
 'submittedAt': '2021-11-16T23:17:51.689+00:00',
 'initialQueueTime': 45054,
 'totalQueueTime': 45054,
 'averageModelLatency': 2814.0,
 'totalModelLatency': 2814.0,
 'elapsedTime': 49311,
 'startingResultSummarizing': '2021-11-16T23:18:40.394+00:00',
 'resultSummarizing': 606,
 'inputSize': 383,
 'results': {'input': {'status': 'SUCCESSFUL',
   'engine': 'model-batch-vpf0yrzjle-0-1-0-859664b44f-flbwn',
   'inputFetching': 2611,
   'outputUploading': None,
   'modelLatency': 2814.0,
   'queueTime': 45054,
   'startTime': '2021-11-16T23:18:36.539+0000',
   'updateTime': '2021-11-16T23:18:40.298+0000',
   'endTime': '2021-11-16T23:18:40.298+0000',
   'results.json': {'data': {'result': {'classPredictions'